<a href="https://colab.research.google.com/github/rallyfranky/my-first-repo/blob/main/hate_speech_competition20221009.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [87]:
! pip install mecab-python3 unidic-lite
import MeCab
import re
from collections import Counter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


前処理

In [88]:
train = pd.read_csv('/content/drive/MyDrive/train.csv')
train.head()
train.tail()

,id,source,text,label
5251,42b5f86b0,news4vip,車じゃなくてもよくない？\nケーブル網を張り巡らせてリフトみたいなのを付けるとか,0
5252,402ce15d9,livejupiter,左やぞ？\n出すなら下水流,0
5253,0739a9fcb,newsplus,日本によって、けんけんガクガクの議論を持たらされた韓国は被害者\n\n日本人がしっかり考えな...,0
5254,d496c7dc8,news4vip,ゴムボート買って、沖まで漕いで行ったら？\n魚の血を塗っておけばサメが食べてくれるよ,0
5255,d1838f4f0,news4vip,男は美化してないだろ？普通にエロい部分や格好悪い所も描かれてる\n女の方は主人公美少女しかい...,0


In [89]:
X = train.iloc[:,2]
y = train.iloc[:,3]
print(X)

0                       まともに相手されてない人との関係なんて\nそんな大事にするものか？
1                  最近はアヘアヘQSマンやない？ ｲｲ!(・∀・)+1-0(・Ａ・)ｲｸﾅｲ!
2         日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ\n甘えるな
3        よくよく思えば川上は配布にしたらとんでもなく有能だよな\nガチャから引いたら圧倒的歓喜レベルやで
4       押井は原作レイプの専門家だから\n原作マンガの真意を誤解させることに関してはプロだが\nそれ...
                              ...                        
5251             車じゃなくてもよくない？\nケーブル網を張り巡らせてリフトみたいなのを付けるとか
5252                                        左やぞ？\n出すなら下水流
5253    日本によって、けんけんガクガクの議論を持たらされた韓国は被害者\n\n日本人がしっかり考えな...
5254           ゴムボート買って、沖まで漕いで行ったら？\n魚の血を塗っておけばサメが食べてくれるよ
5255    男は美化してないだろ？普通にエロい部分や格好悪い所も描かれてる\n女の方は主人公美少女しかい...
Name: text, Length: 5256, dtype: object


In [90]:
tokeData=[]
select = ["動詞","形容詞","名詞"]
mecab = MeCab.Tagger('-Owakati')
mecab.parse('')
#分かち書きと品詞の種類（select）に合わせた絞り込みの関数作成。
def wakati_text(text):
    # 分けてノードごとにする
    node = mecab.parseToNode(text)
    terms = []

    while node:

        # 単語
        term = node.surface
        # 品詞
        pos = node.feature.split(',')[0]
        # もし品詞が条件と一致してたら
        if pos in select:
            terms.append(term)
        node = node.next
    # スペースを入れながら結合する。
    text_result = ' '.join(terms)
    return text_result

#行数分関数を実行し、tokeDataのリストに格納
for i in range(len(X)):
  tmp = wakati_text(X[i])
  tokeData.append(tmp)

In [91]:
#抽出された変数を見ると大量の数字、正規表現で数字指定し変換。
X_wakati=[]
for j in range(len(X)):
  tmp2 = re.sub(r'[0-9]+','',tokeData[j])
  X_wakati.append(tmp2)

In [92]:
X_wakati

['相手 さ 人 関係 する もの',
 '最近 アヘアヘ QS マン ない ｲｲ   Ａ ｲｸﾅｲ',
 '日本 し 生まれ 無能 学歴 分かっ 日本 権利 剥奪 し 追放 す 甘える',
 '思え 川上 配布 し なく 有能 引い 圧倒 歓喜 レベル',
 '押井 原作 レイプ 専門 原作 マンガ 真意 誤解 さ こと 関し プロ 以外 取り柄 ない',
 'マジレス する 野党 任せ ドイツ し 場合 税金 難民 養う こと なる 生活 保護 話 ない 金 話 する 視野 広く せ ｗ',
 '退治 わけ 殺さ 無実 人間 言い する 退治 さ',
 'D 柳 明治 京田 日大 応援 する 予定 両方 シード 来 欲しい',
 'キャラクター いっ もの あり 物事 キャラクター する 感じ イメージ 人 それぞれ',
 '都内 幹線 道路 一角 い 百 円 醤油 効い 海苔巻き 餅 売っ くれ オジ 旨かっ',
 '女 精子 なく 男 卵子 なく 産める おもう 段階 倫理 人間 進歩 疎外 し',
 '付加 機能 ない 安い 機種 数字 ボタン 押し 電話 掛ける ヤツ',
 '神 愛 捧げる 人生 良い 転機 訪れ 大体 中 神社 神道 絡ん くる とき',
 'キラ 付け やっ',
 '予定 ある 気 乗ら 普通 断れ いい 今月 金 無い 理由 断る 引く 飲み し  万 円 以下 話',
 'いう ところ 魅力 思っ い うそ 本当 こと みんな 議論 する すごく',
 'プロ なく 連れ 女の子 何 捕まえれる 競争 し し w 世間 言う チャラ 男',
 'わかっ ホーム ラン 打て 稼げる 凄い 思わ',
 '韓国 辛い もの 食べ 消化 器官 癌 発症 率 半端 ねー',
 '内 ホウ酸 団子 無理 なっ 笑えん ゴキブリホイホイ ある',
 'ファースト エイド リアル 救急 箱 www コンパス ポーチ 忍ばし おく',
 '生身 ドス 攻撃 言う やり',
 '黒人 なまり ラップ 日本 ラップ し 笑',
 '連中 法 従っ 動い 奴 せい する 門違い 野放し し なきゃ いう 法律 作る ない',
 'フェミニスト 思っ いわ こと なかっ',
 'ネット 見 声掛け 不快 意見 多い 事 知ら',
 'java 本 読ん 

In [93]:
#ベクトル化
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
count = bow.fit_transform(X_wakati)
vec = count.toarray()
name = bow.get_feature_names_out()
pd.DataFrame(vec, columns=name)

,aa,aaf,ab,abc,ac,ada,age,ai,aiib,ak,...,ﾌﾞﾘﾌﾞﾘﾌﾞﾘﾌﾞﾘｭﾘｭﾘｭﾘｭﾘｭﾘｭ,ﾎﾝﾄ,ﾎﾟｯﾌﾟｱﾝﾄﾞｹﾞｯﾂｰ,ﾎﾟｹ,ﾒﾝﾄﾞｸｾ,ﾜｶﾓﾉ,ﾜﾛﾀ,ﾞﾁ,ﾞﾌ,ﾟヮﾟﾉ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5251,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5252,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5253,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5254,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
X_vec = pd.DataFrame(vec, columns=name)

In [95]:
#まずはl2
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
X_vec_std = std.fit_transform(X_vec)
lr = LogisticRegression(C=10,penalty='l2',random_state=1)

In [96]:
#ホールドアウト
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_vec_std, y, test_size=0.2, stratify=y, random_state=1)

In [97]:
lr.fit(X_train, y_train)

LogisticRegression(C=10, random_state=1)

In [98]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(estimator=lr,X=X_train,y=y_train,cv=10,n_jobs=1)

In [99]:
print('accuracy is %s' % scores)

accuracy is [0.9263658  0.92874109 0.9263658  0.93111639 0.93095238 0.92380952
 0.93809524 0.93333333 0.93333333 0.93809524]


In [100]:
print('accuracy is %.3f +- %.3f' % (np.mean(scores), np.std(scores)))

accuracy is 0.931 +- 0.005


In [101]:
y_pred = lr.predict(X_val)

In [102]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score, f1_score

print('Precision is %.3f' % precision_score(y_true=y_val, y_pred=y_pred))
print('Recall is %.3f' % recall_score(y_true=y_val, y_pred=y_pred))
print('f1 is %.3f' % f1_score(y_true=y_val, y_pred=y_pred))

Precision is 0.556
Recall is 0.246
f1 is 0.341


In [103]:
from sklearn.metrics import confusion_matrix
confmat = confusion_matrix(y_true=y_val, y_pred=y_pred)
print(confmat)

[[979  12]
 [ 46  15]]


In [104]:
X_train, X_val, y_train, y_val = train_test_split(X_wakati, y, test_size=0.2, stratify=y, random_state=1)

In [105]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
tfidfvec = TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None)

#パイプライン
param_grid = [{'vect__ngram_range':[(1,1)],'clf__penalty':['l1','l2'],'clf__C':[1.0,10.0,100.0]},
              {'vect__ngram_range':[(1,1)],'vect__use_idf':[False],'vect__norm':[None],'clf__penalty':['l1','l2'],'clf__C':[1.0,10.0,100.0]}]
lr_tfidfvec = Pipeline([('vect',tfidfvec),('clf',LogisticRegression(random_state=0,solver='liblinear'))])
gs_lr_tfidf = GridSearchCV(lr_tfidfvec,param_grid,scoring='f1',cv=5,verbose=2,n_jobs=1)
gs_lr_tfidf.fit(X_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1); total time=   0.1s
[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1); total time=   0.1s
[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1); total time=   0.1s
[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1); total time=   0.1s
[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1); total time=   0.1s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1); total time=   0.1s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1); total time=   0.1s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1); total time=   0.1s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1); total time=   0.1s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1); total time=   0.1s


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


[CV] END clf__C=10.0, clf__penalty=l1, vect__ngram_range=(1, 1); total time=   2.9s


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


[CV] END clf__C=10.0, clf__penalty=l1, vect__ngram_range=(1, 1); total time=   2.8s


...

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


[CV] END clf__C=10.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__norm=None, vect__use_idf=False; total time=   4.2s
[CV] END clf__C=10.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__norm=None, vect__use_idf=False; total time=   0.1s
[CV] END clf__C=10.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__norm=None, vect__use_idf=False; total time=   0.1s


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


[CV] END clf__C=10.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__norm=None, vect__use_idf=False; total time=   4.6s
[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__use_idf=False; total time=   0.1s
[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__use_idf=False; total time=   0.1s
[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__use_idf=False; total time=   0.1s
[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__use_idf=False; total time=   0.1s
[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__use_idf=False; total time=   0.1s
[CV] END clf__C=100.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__norm=None, vect__use_idf=False; total time=   0.1s
[CV] END clf__C=100.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__norm=None, vect__use_idf=False; total time=   0.1s
[CV] END clf__C=100.0,

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)]},
                         {'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)], 'vect__norm': [None],
                          'vect__use_idf': [False]}],
             scoring='f1', verbose=2)

In [106]:
print('paramater: %s' % gs_lr_tfidf.best_params_)

paramater: {'clf__C': 10.0, 'clf__penalty': 'l1', 'vect__ngram_range': (1, 1), 'vect__norm': None, 'vect__use_idf': False}


In [107]:
clf = gs_lr_tfidf.best_estimator_
print('test f1 is: %.3f' % clf.score(X_val, y_val))

test f1 is: 0.943
